In [2]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from config import DATABASE_URI
from models import Security
from sqlalchemy import Column, Table
from scrapping_sources.Macrotrend import Macrotrend

In [35]:
engine = create_engine(DATABASE_URI)
Session = sessionmaker(bind=engine)
s = Session()

In [3]:
q = s.query(Security.ticker).all()

In [160]:
[i for i in range(0,75,10)]
# COMPARE WITH MYSQL OUTPUT. MAKE SURE THEY ARE THE SAME

[0, 10, 20, 30, 40, 50, 60, 70]

In [166]:
my_list = [i for i in range(0,75)]

In [168]:
[my_list[i:i + 10] for i in range(0, len(my_list), 10)]

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
 [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
 [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
 [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
 [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
 [70, 71, 72, 73, 74]]

In [174]:
def chunks(lst, n):
    chunk_list = []
    for i in range(0, len(lst), n):
        chunk_list.append(lst[i:i + n])
    return chunk_list

In [175]:
chunks(my_list, 10)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
 [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
 [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
 [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
 [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
 [70, 71, 72, 73, 74]]

In [17]:
s.close_all()

/tmp/ipykernel_4768/1419930820.py:1: SADeprecationWarning: The Session.close_all() method is deprecated and will be removed in a future release.  Please refer to session.close_all_sessions(). (deprecated since: 1.3)
  s.close_all()


In [5]:
r = s.query(Security.ticker).all()

In [6]:
tickers =[i[0] for i in r][:100]

In [20]:
scrapper = Macrotrend()
scrapper.generate_statement_table_multi_threading(tickers, "balance-sheet", "annual")

ValueError: No objects to concatenate

In [22]:
import time
import pandas as pd
import requests, json, re
from bs4 import BeautifulSoup as bs
import datetime
import concurrent.futures
from itertools import repeat



class Macrotrend():

    # def __init__(self): #self, ticker=None, statement=None, time_format=None, **kwargs):
    #     # super().__init__(**kwargs)
    #     # self.ticker         = ticker
    #     # self.statement      = statement
    #     # self.time_format    = time_format
    #     pass
        
    def get_statement(self, ticker, statement, time_format):

        statement_data = None

        if time_format == 'annual':

            try:

                r = requests.get('https://www.macrotrends.net/stocks/charts/' + ticker + '/' + ticker + '/' + statement)
                print(ticker, r)
                p = re.compile(r'var originalData = (.*);')
                p2 = re.compile(r'datafields:[\s\S]+(\[[\s\S]+?\]),')
                p3 = re.compile(r'\d{4}-\d{2}-\d{2}')
                data = json.loads(p.findall(r.text)[0])
                s = re.sub('\r|\n|\t|\s','',p2.findall(r.text)[0])
                fields = p3.findall(s)
                fields.insert(0, 'field_name') # only headers of interest.
                results = []

                for item in data: #loop initial list of dictionaries
                    row = {}
                    for f in fields: #loop keys of interest to extract from current dictionary
                        if f == 'field_name':  #this is an html value field so needs re-parsing
                            soup2 = bs(item[f],'lxml')
                            row[f] = soup2.select_one('a,span').text
                        else:
                            row[f] = item[f]
                    results.append(row)

                # return pd.DataFrame(results, columns = fields)
                statement_data = pd.DataFrame(results, columns = fields)

                return statement_data
        
            except:
                print(f"No data available for {ticker}, {statement}, annual")
                pass

            pass

        else:

            try:

                r = requests.get(f'https://www.macrotrends.net/stocks/charts/' + ticker + '/' + ticker + '/' + statement)
                company_name = r.url.split('/')[6]
                r = requests.get(f'https://www.macrotrends.net/stocks/charts/'+ ticker + '/' + company_name + '/' + statement + '?freq=Q')
                print(ticker, r)
                p = re.compile(r'var originalData = (.*);')
                p2 = re.compile(r'datafields:[\s\S]+(\[[\s\S]+?\]),')
                p3 = re.compile(r'\d{4}-\d{2}-\d{2}')
                data = json.loads(p.findall(r.text)[0])
                s = re.sub('\r|\n|\t|\s','',p2.findall(r.text)[0])
                fields = p3.findall(s)
                fields.insert(0, 'field_name') # only headers of interest.
                results = []

                for item in data: #loop initial list of dictionaries
                    row = {}
                    for f in fields: #loop keys of interest to extract from current dictionary
                        if f == 'field_name':  #this is an html value field so needs re-parsing
                            soup2 = bs(item[f],'lxml')
                            row[f] = soup2.select_one('a,span').text
                        else:
                            row[f] = item[f]
                    results.append(row)

                # return pd.DataFrame(results, columns = fields)
                statement_data = pd.DataFrame(results, columns = fields)
                return statement_data
    
            except:
                print(f"No data available for {ticker}, {statement}, quarterly")
                pass

    def move_column(self, df, column, pos):
    
        col = df[column]
        df.drop(columns=[column],inplace = True)
        df.insert(pos, column, col)
        
        return df
    
    def generate_statement_key(self, statement, time_format):

        report_formats  = {'quarterly'  : 'Q',
                            'annual'    : 'A'}
        statements      = {'income-statement'       :'IS',
                            'balance-sheet'         :'BS',
                            'cash-flow-statement'   :'CF',
                            'financial-ratios'      :'R'}

        statement_key = statements[statement] + '-' + report_formats[time_format]

        return statement_key

    def arrange_data(self, ticker, statement, time_format):

        data_dict = []
        # no_data_tick = []
        report_formats = {'quarterly' : 'Q',
                        'annual': 'A'}
        statements = {'income-statement':'IS',
                    'balance-sheet':'BS',
                    'cash-flow-statement':'CF',
                    'financial-ratios':'R'}


        df = self.get_statement(ticker, statement, time_format)

        if isinstance(df, pd.DataFrame):

            for i in df.columns[1:]:

                keys = df[df.columns[0]]
                data = df[i].values

                data = {k:v for k,v in zip(keys, data)}
                date = i

                data['date'] = date
                data['ticker'] = ticker
                data['statement_format'] = statements[statement] + '-' + report_formats[time_format]

                data_dict.append(data)

            df = pd.DataFrame(data_dict)
            df = self.move_column(df, 'date', 0)
            df = self.move_column(df, 'ticker', 1)
            df = self.move_column(df, 'statement_format', 2)

            my_series = []

            line_item = []
            amount = []
            date = []
            ticker = []
            security_id = []
            statement_format = []

            for i in df.iterrows():

                serie = pd.Series(i)[1]

                my_series.append(serie)
                [statement_format.append(serie[2]) for i in range(len(serie.index.values[4:]))]
                # [security_id.append(serie[3]) for i in range(len(serie.index.values[4:]))]
                [line_item.append(i) for i in  serie.index.values[4:]]
                [amount.append(i) for i in serie.values[4:]]
                [date.append(serie[0]) for i in  range(len(serie.index.values[4:]))]
                [ticker.append(serie[1]) for i in range(len(serie.index.values[4:]))]

            data = pd.DataFrame([date, statement_format, ticker, security_id,line_item, amount]).T
            data.columns = ['date','statement','ticker','security_id','line_item','amount']
            return data

        else:
            
            pass

    def generate_statement_table_multi(self, ticker_list, statement, time_format):
        start = time.perf_counter()
        table   = []
        # no_income_statement_list = []

        for i in ticker_list:
            df = self.arrange_data(i, statement, time_format)

            if isinstance(df, pd.DataFrame):
                table.append(df)
            else:
                pass
        
        table_concat = pd.concat(table)
        table_concat['amount'] =  pd.to_numeric(table_concat['amount'])
        finish = time.perf_counter()
        print(finish-start)
        return table_concat

    def generate_statement_table_multi_threading(self, ticker_list, statement, time_format):
        start = time.perf_counter()
        table = []

        def create_table(ticker, statement, time_format):

            df = self.arrange_data(ticker, statement, time_format)
            if isinstance(df, pd.DataFrame):
                table.append(df)


        with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:

            executor.map(create_table, ticker_list, repeat(statement), repeat(time_format))

        table_concat = pd.concat(table)
        table_concat['amount'] =  pd.to_numeric(table_concat['amount'])
        finish = time.perf_counter()
        print(finish-start)
        return table_concat

    def generate_statement_table(self, ticker, statement, time_format):

        pass
        # TRY SIMPLY GOING WITH THE OLD SCRIPT BY UPLOADING IN CHUNKS
    
    def latest_ending_period_available(self, ticker, statement, time_format):

        try:
            df = self.get_statement(ticker, statement, time_format)
            raw_date = df.columns[1]
            converted_date = datetime.datetime.strptime(str(raw_date), '%Y-%m-%d').strftime("%Y-%m")
        except:
            converted_date = None

        return converted_date

scrapper = Macrotrend()

print(scrapper.latest_ending_period_available('AAPL', 'balance-sheet', 'quarterly'))
    

AAPL <Response [200]>
2021-06


In [23]:
scrapper = Macrotrend()

In [24]:
scrapper.generate_statement_table_multi_threading(tickers, "balance-sheet", "quarterly")

A <Response [200]>
AA <Response [200]>
AACIU <Response [404]>
No data available for AACIU, balance-sheet, quarterly
AADR <Response [404]>
No data available for AADR, balance-sheet, quarterly
AAIC <Response [200]>
AADI <Response [200]>
AACAACG <Response [200]> <Response [200]>

AAAU <Response [404]>
No data available for AAAU, balance-sheet, quarterly
AAL <Response [200]>
AAMC <Response [200]>
AANAAME  <Response [200]>
<Response [200]>
AAOI <Response [200]>
AAON <Response [200]>
AAPL <Response [200]>
AAP <Response [200]>
AAQC <Response [200]>
AAT <Response [200]>
AATC <Response [200]>
AAU <Response [200]>
AAWW <Response [200]>
AAXJ <Response [404]>
No data available for AAXJ, balance-sheet, quarterly
ABB <Response [200]>
AB <Response [200]>
ABBV <Response [200]>
ABCL <Response [200]>
ABCB <Response [200]>
ABC <Response [200]>
ABEQ <Response [404]>
No data available for ABEQ, balance-sheet, quarterly
ABCM <Response [200]>
ABEO <Response [200]>
ABEV <Response [200]>
ABGI <Response [200]>


,date,statement,ticker,security_id,line_item,amount
0,2021-07-31,BS-Q,A,None,Receivables,1122.000
1,2021-07-31,BS-Q,A,None,Inventory,818.000
2,2021-07-31,BS-Q,A,None,Pre-Paid Expenses,NaN
3,2021-07-31,BS-Q,A,None,Other Current Assets,264.000
4,2021-07-31,BS-Q,A,None,Total Current Assets,3632.000
...,...,...,...,...,...,...
127,2019-12-31,BS-Q,ACVA,None,Retained Earnings (Accumulated Deficit),-125.958
128,2019-12-31,BS-Q,ACVA,None,Comprehensive Income,NaN
129,2019-12-31,BS-Q,ACVA,None,Other Share Holders Equity,-0.001
130,2019-12-31,BS-Q,ACVA,None,Share Holder Equity,205.326


In [9]:
table = []

def create_table(ticker, statement, time_format):

    df = scrapper.arrange_data(ticker, statement, time_format)
    if isinstance(df, pd.DataFrame):
        table.append(df)


with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:

    executor.map(create_table, tickers, repeat('balance-sheet'), repeat("annual"))

# table
table_concat = pd.concat(table)
table_concat['amount'] =  pd.to_numeric(table_concat['amount'])


AAC <Response [200]>
AA <Response [200]>AAAU
 <Response [404]>
No data available for AAAU, balance-sheet, annual
AADR <Response [404]>
No data available for AADR, balance-sheet, annual
A <Response [200]>
AACIUAAIC <Response [200]>
AAL <Response [200]>
 <Response [404]>
No data available for AACIU, balance-sheet, annual
AADI <Response [200]>
AACG <Response [200]>
AAMC <Response [200]>AAME <Response [200]>

AAN <Response [200]>
AAOI <Response [200]>
AAPL <Response [200]>
AAON <Response [200]>
AAP <Response [200]>
AAQC <Response [200]>
No data available for AAQC, balance-sheet, annual
AAT AATC<Response [200]> <Response [200]>

AAXJ <Response [404]>
No data available for AAXJ, balance-sheet, annual
AAU <Response [200]>
ABB <Response [200]>
ABAAWW  <Response [200]><Response [200]>

ABBV <Response [200]>
ABCB <Response [200]>
ABCL <Response [200]>
ABC <Response [200]>
ABCM <Response [200]>
ABEQ <Response [404]>
No data available for ABEQ, balance-sheet, annual
ABEO <Response [200]>
ABEV <Res

In [12]:
table_concat

,date,statement,ticker,security_id,line_item,amount
0,2020-12-31,BS-A,AA,None,Receivables,556.000
1,2020-12-31,BS-A,AA,None,Inventory,1398.000
2,2020-12-31,BS-A,AA,None,Pre-Paid Expenses,290.000
3,2020-12-31,BS-A,AA,None,Other Current Assets,21.000
4,2020-12-31,BS-A,AA,None,Total Current Assets,4520.000
...,...,...,...,...,...,...
39,2019-12-31,BS-A,ACVA,None,Retained Earnings (Accumulated Deficit),-125.958
40,2019-12-31,BS-A,ACVA,None,Comprehensive Income,-0.001
41,2019-12-31,BS-A,ACVA,None,Other Share Holders Equity,NaN
42,2019-12-31,BS-A,ACVA,None,Share Holder Equity,205.326


In [45]:
scrapper.get_statement('AAPL', 'balance-sheet', 'annual')

AAPL <Response [200]>
No data available for AAPL, balance-sheet, annual


In [31]:
s.query(Security.ticker).filter_by(ticker="jlkkj").first()[0]

TypeError: 'NoneType' object is not subscriptable

In [36]:
from models import Base

In [51]:
s.query(Base.metadata.tables['income_statement_annual'].columns.ticker).filter_by(ticker='A').all()

[('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 

In [52]:
s.close_all()

/tmp/ipykernel_68765/1419930820.py:1: SADeprecationWarning: The Session.close_all() method is deprecated and will be removed in a future release.  Please refer to session.close_all_sessions(). (deprecated since: 1.3)
  s.close_all()


In [3]:
A = ('a','b','c')
B = ('a','b','c')
C = ('d','e','f')
D = ('g','h','i')

In [4]:
my_list = [A,B,C]

In [6]:
set_1 = set(my_list)
set_2 = set([C,D])

In [8]:
len(set_1-set_2)

1